In [26]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# encodage des mots en Int
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau


from keras.layers.merge import dot
from keras.models import Model


# specifically for deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG
 
# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
#import cv2 # erreur import
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

#TL pecific modules
from keras.applications.vgg16 import VGG16

# pour parser le fichier train.json
import json

In [40]:
with open('train.json', 'r') as f:
    datas = json.load(f) # id, cuisine, ingredients[]

csv = []
for data in datas:
    for ingredient in data['ingredients']:
        tmp = dict()
        tmp['id'] = str(data['id'])
        tmp['cuisine'] = data['cuisine']
        tmp['ingredient'] = ingredient
        csv.append(tmp)

train=pd.DataFrame(csv)

In [55]:
df=train.copy()

In [54]:
df.head()

,cuisine,id,ingredient
0,6,10259,5222
1,6,10259,956
2,6,10259,3033
3,6,10259,2884
4,6,10259,4569


In [53]:
le_cuisine = LabelEncoder()
le_ingredient = LabelEncoder()

df['cuisine'] = le_cuisine.fit_transform(df['cuisine'])
df['ingredient'] = le_ingredient.fit_transform(df['ingredient'])

# Pour inverser l'encodage il faudra utiliser ces deux lignes :
df['cuisine'] = le_cuisine.inverse_transform(df['cuisine'])
df['ingredient'] = le_ingredient.inverse_transform(df['ingredient'])

df.head()

,cuisine,id,ingredient
0,6,10259,5222
1,6,10259,956
2,6,10259,3033
3,6,10259,2884
4,6,10259,4569


In [49]:
df['cuisine'].unique()

array([ 6, 16,  4,  7, 10, 17,  9, 13,  3,  1, 18, 19,  2,  0,  5, 11,  8,
       12, 14, 15])

In [50]:
len(df['cuisine'].unique())

20

In [51]:
df['ingredient'].unique()

array([5222,  956, 3033, ..., 3818, 6136, 2090])

In [52]:
len(df['ingredient'].unique())

6714